In [1]:
import sys
import os
sys.path.append('../')
sys.path.append('../WD_models')

### General
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from astropy.io import fits
from astropy.table import Table, Column, MaskedColumn, join, unique
from astropy.table import *

from tqdm import tqdm
import pandas as pd
#import WD_models

In [2]:
#from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import table
import astropy.units as u

wd_c1 = Table.read('sdss5/spAll-v6_1_0.fits')


In [3]:
wd_c1[0]

PROGRAMNAME,SURVEY,FIELDQUALITY,FIELDSN2,EXP_DISP_MED,FIBERID_LIST,LAMBDA_EFF,BLUEFIBER,ZOFFSET,XFOCAL,YFOCAL,CALIBFLUX,CALIBFLUX_IVAR,GAIA_BP,GAIA_RP,GAIA_G,CADENCE,FIRSTCARTON,CARTON_TO_TARGET_PK,RACAT,DECCAT,COORD_EPOCH,PMRA,PMDEC,PARALLAX,CATALOGID,CATALOGID_V0,CATALOGID_V0P5,GAIA_ID_DR2,FIBER2MAG,PSFMAG,CATDB_MAG,OPTICAL_PROV,OBS,FIELD,DESIGNS,CONFIGS,NEXP,EXPTIME,AIRMASS,SEEING20,SEEING50,SEEING80,ASSIGNED,ON_TARGET,VALID,HEALPIX,HEALPIXGRP,HEALPIX_PATH,MJD_FINAL,MJD_LIST,TAI_LIST,FIELDSNR2G_LIST,FIELDSNR2R_LIST,FIELDSNR2I_LIST,RA_LIST,DEC_LIST,DELTA_RA_LIST,DELTA_DEC_LIST,MOON_DIST,MOON_PHASE,EBV,EBV_TYPE,WISE_MAG,TWOMASS_MAG,GUVCAT_MAG,FIBER_OFFSET,SPEC_FILE,XCSAO_RV,XCSAO_ERV,XCSAO_RXC,XCSAO_TEFF,XCSAO_ETEFF,XCSAO_LOGG,XCSAO_ELOGG,XCSAO_FEH,XCSAO_EFEH,SDSSV_BOSS_TARGET0,MJD,TARGET_INDEX,RUN2D,RUN1D,OBJTYPE,FIBER_RA,FIBER_DEC,PLUG_RA,PLUG_DEC,CLASS,SUBCLASS,Z,Z_ERR,RCHI2,DOF,RCHI2DIFF,TFILE,TCOLUMN,NPOLY,THETA,VDISP,VDISP_ERR,VDISPZ,VDISPZ_ERR,VDISPCHI2,VDISPNPIX,VDISPDOF,WAVEMIN,WAVEMAX,WCOVERAGE,ZWARNING,SN_MEDIAN,SN_MEDIAN_ALL,CHI68P,FRACNSIGMA,FRACNSIGHI,FRACNSIGLO,SPECTROFLUX,SPECTROFLUX_IVAR,SPECTROSYNFLUX,SPECTROSYNFLUX_IVAR,SPECTROSKYFLUX,ANYANDMASK,ANYORMASK,SPEC1_G,SPEC1_R,SPEC1_I,SPEC2_G,SPEC2_R,SPEC2_I,Z_NOQSO,Z_ERR_NOQSO,ZNUM_NOQSO,ZWARNING_NOQSO,CLASS_NOQSO,SUBCLASS_NOQSO,RCHI2DIFF_NOQSO,SPECOBJID
bytes13,bytes13,bytes4,float32,float64,bytes111,float32,int32,float32,bytes251,bytes251,float32[5],float32[5],float32,float32,float32,bytes13,bytes48,bytes22,float64,float64,float32,float32,float32,float32,int64,int64,int64,int64,float32[5],float32[5],float32[5],bytes26,bytes3,int32,bytes125,bytes167,int16,int16,float32,float32,float32,float32,bytes55,bytes55,bytes55,int32,int16,bytes76,float64,bytes167,bytes307,bytes139,bytes147,bytes152,bytes307,bytes279,bytes111,bytes111,bytes139,bytes139,float32,bytes11,float32[4],float32[3],float32[2],float32,bytes42,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,int32,int32,bytes6,bytes6,bytes16,float64,float64,float64,float64,bytes6,bytes21,float32,float32,float32,int32,float32,bytes24,int32[10],int32,float32[10],float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,int32,float32[5],float32,float32,float32[10],float32[10],float32[10],float32[5],float32[5],float32[5],float32[5],float32[5],int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,bytes6,bytes21,float32,int64
RM,BHM,good,7.287694,1.049497686772183,424 424,5400.0,0,0.0,-131.001 -131.001,-270.826 -270.826,0.0 .. 116.69114,0.0 .. 0.0,17.9881,17.045,18.1457,plates,bhm_rm_known-spec,-999,35.10384601646094,-6.292128211174298,2000.0,0.32969478,1.2853869,-0.052332718,4375786564,4375786564,0,2487098123823599616,-9.99999 .. 18.1579,-10.797755 .. 17.334406,-9.99999 .. 18.1579,fiber2mag,APO,15000,,15000 15000,2,1800,1.2987098,1.483993,1.6015084,1.7554159,1 1,1 1,1 1,108850,108,$MWM_HEALPIX/108/108850/boss/v6_1_0/spec-015000-59146-4375786564.fits,59146.33513143499,59146 59146,5110242925 5110243926,4.13 3.16,10.94 8.40,8.01 6.04,35.103846 35.103846,-6.292128 -6.292128,0.0 0.0,0.0 0.0,79.4 79.2,0.58 0.58,0.026325047,SFD,13.365 .. 7.285,15.866 .. 14.577,18.7659 .. 18.5614,0.0,spec-015000-59146-4375786564.fits,372.47113,25.287928,1.6334513,4396.9653,1863.9761,3.54361,1.8261423,-0.27068064,0.75969946,25769804288,59146,1,v6_1_0,v6_1_0,QSO,35.103846,-6.2921282,35.103845999999976,-6.2921282,QSO,BROADLINE,0.13961641,3.292187e-05,2.3595521,4513,6.737322,spEigenQSO-58596.fits,0 .. 9,0,0.02268109 .. -0.034858134,0.0,0.0,0.0,0.0,0.0,0.0,0,3626.6028,10332.371,0.4523,64,6.2959604 .. 18.618938,18.596743,1.1759084,0.39221755 .. 0.0022109218,0.21799691 .. 0.0017687376,0.17422065 .. 0.0004421844,19.684925 .. 120.76659,0.24994858 .. 0.71659154,43.932167 .. 116.8185,2.8173938 .. 1.4498618,20.8105 .. 152.26627,88670208,256835584,7.287694,19.336294,14.045464,0.0,0.0,0.0,0.139387,2.31031e-05,2,0,GALAXY,,1.9262252,-1558245042737524736


In [4]:
wd_c1

PROGRAMNAME,SURVEY,FIELDQUALITY,FIELDSN2,EXP_DISP_MED,FIBERID_LIST,LAMBDA_EFF,BLUEFIBER,ZOFFSET,XFOCAL,YFOCAL,CALIBFLUX,CALIBFLUX_IVAR,GAIA_BP,GAIA_RP,GAIA_G,CADENCE,FIRSTCARTON,CARTON_TO_TARGET_PK,RACAT,DECCAT,COORD_EPOCH,PMRA,PMDEC,PARALLAX,CATALOGID,CATALOGID_V0,CATALOGID_V0P5,GAIA_ID_DR2,FIBER2MAG,PSFMAG,CATDB_MAG,OPTICAL_PROV,OBS,FIELD,DESIGNS,CONFIGS,NEXP,EXPTIME,AIRMASS,SEEING20,SEEING50,SEEING80,ASSIGNED,ON_TARGET,VALID,HEALPIX,HEALPIXGRP,HEALPIX_PATH,MJD_FINAL,MJD_LIST,TAI_LIST,FIELDSNR2G_LIST,FIELDSNR2R_LIST,FIELDSNR2I_LIST,RA_LIST,DEC_LIST,DELTA_RA_LIST,DELTA_DEC_LIST,MOON_DIST,MOON_PHASE,EBV,EBV_TYPE,WISE_MAG,TWOMASS_MAG,GUVCAT_MAG,FIBER_OFFSET,SPEC_FILE,XCSAO_RV,XCSAO_ERV,XCSAO_RXC,XCSAO_TEFF,XCSAO_ETEFF,XCSAO_LOGG,XCSAO_ELOGG,XCSAO_FEH,XCSAO_EFEH,SDSSV_BOSS_TARGET0,MJD,TARGET_INDEX,RUN2D,RUN1D,OBJTYPE,FIBER_RA,FIBER_DEC,PLUG_RA,PLUG_DEC,CLASS,SUBCLASS,Z,Z_ERR,RCHI2,DOF,RCHI2DIFF,TFILE,TCOLUMN,NPOLY,THETA,VDISP,VDISP_ERR,VDISPZ,VDISPZ_ERR,VDISPCHI2,VDISPNPIX,VDISPDOF,WAVEMIN,WAVEMAX,WCOVERAGE,ZWARNING,SN_MEDIAN,SN_MEDIAN_ALL,CHI68P,FRACNSIGMA,FRACNSIGHI,FRACNSIGLO,SPECTROFLUX,SPECTROFLUX_IVAR,SPECTROSYNFLUX,SPECTROSYNFLUX_IVAR,SPECTROSKYFLUX,ANYANDMASK,ANYORMASK,SPEC1_G,SPEC1_R,SPEC1_I,SPEC2_G,SPEC2_R,SPEC2_I,Z_NOQSO,Z_ERR_NOQSO,ZNUM_NOQSO,ZWARNING_NOQSO,CLASS_NOQSO,SUBCLASS_NOQSO,RCHI2DIFF_NOQSO,SPECOBJID
bytes13,bytes13,bytes4,float32,float64,bytes111,float32,int32,float32,bytes251,bytes251,float32[5],float32[5],float32,float32,float32,bytes13,bytes48,bytes22,float64,float64,float32,float32,float32,float32,int64,int64,int64,int64,float32[5],float32[5],float32[5],bytes26,bytes3,int32,bytes125,bytes167,int16,int16,float32,float32,float32,float32,bytes55,bytes55,bytes55,int32,int16,bytes76,float64,bytes167,bytes307,bytes139,bytes147,bytes152,bytes307,bytes279,bytes111,bytes111,bytes139,bytes139,float32,bytes11,float32[4],float32[3],float32[2],float32,bytes42,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,int32,int32,bytes6,bytes6,bytes16,float64,float64,float64,float64,bytes6,bytes21,float32,float32,float32,int32,float32,bytes24,int32[10],int32,float32[10],float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,int32,float32[5],float32,float32,float32[10],float32[10],float32[10],float32[5],float32[5],float32[5],float32[5],float32[5],int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,bytes6,bytes21,float32,int64
RM,BHM,good,7.287694,1.049497686772183,424 424,5400.0,0,0.0,-131.001 -131.001,-270.826 -270.826,0.0 .. 116.69114,0.0 .. 0.0,17.9881,17.045,18.1457,plates,bhm_rm_known-spec,-999,35.10384601646094,-6.292128211174298,2000.0,0.32969478,1.2853869,-0.052332718,4375786564,4375786564,0,2487098123823599616,-9.99999 .. 18.1579,-10.797755 .. 17.334406,-9.99999 .. 18.1579,fiber2mag,APO,15000,,15000 15000,2,1800,1.2987098,1.483993,1.6015084,1.7554159,1 1,1 1,1 1,108850,108,$MWM_HEALPIX/108/108850/boss/v6_1_0/spec-015000-59146-4375786564.fits,59146.33513143499,59146 59146,5110242925 5110243926,4.13 3.16,10.94 8.40,8.01 6.04,35.103846 35.103846,-6.292128 -6.292128,0.0 0.0,0.0 0.0,79.4 79.2,0.58 0.58,0.026325047,SFD,13.365 .. 7.285,15.866 .. 14.577,18.7659 .. 18.5614,0.0,spec-015000-59146-4375786564.fits,372.47113,25.287928,1.6334513,4396.9653,1863.9761,3.54361,1.8261423,-0.27068064,0.75969946,25769804288,59146,1,v6_1_0,v6_1_0,QSO,35.103846,-6.2921282,35.103845999999976,-6.2921282,QSO,BROADLINE,0.13961641,3.292187e-05,2.3595521,4513,6.737322,spEigenQSO-58596.fits,0 .. 9,0,0.02268109 .. -0.034858134,0.0,0.0,0.0,0.0,0.0,0.0,0,3626.6028,10332.371,0.4523,64,6.2959604 .. 18.618938,18.596743,1.1759084,0.39221755 .. 0.0022109218,0.21799691 .. 0.0017687376,0.17422065 .. 0.0004421844,19.684925 .. 120.76659,0.24994858 .. 0.71659154,43.932167 .. 116.8185,2.8173938 .. 1.4498618,20.8105 .. 152.26627,88670208,256835584,7.287694,19.336294,14.045464,0.0,0.0,0.0,0.139387,2.31031e-05,2,0,GALAXY,,1.9262252,-1558245042737524736
RM,BHM,good,7.287694,7.4948

In [5]:
def strip(catalog):  
    drop_cols = []
    keep_cols = ['CATALOGID', 'FIBER_RA', 'FIBER_DEC', 'MJD', 'FIELD', 'FIELDSN2', 'SPEC_FILE', 'OBS']
        
    for key in catalog.keys():
        if key not in keep_cols:
            drop_cols.append(key)
                 
    catalog.remove_columns(drop_cols)
    
    return catalog

wd_c1 = strip(wd_c1)
ms_c1 = wd_c1.copy()

In [6]:
print(wd_c1.keys())

['FIELDSN2', 'CATALOGID', 'OBS', 'FIELD', 'SPEC_FILE', 'MJD', 'FIBER_RA', 'FIBER_DEC']


In [7]:
print(ms_c1.keys())

['FIELDSN2', 'CATALOGID', 'OBS', 'FIELD', 'SPEC_FILE', 'MJD', 'FIBER_RA', 'FIBER_DEC']


In [8]:
catalog = Table.read('data/elbadry.fits.gz')

catalog = catalog[catalog['binary_type'] == b'WDMS']

In [9]:
def wd_sep(bp_rp):
    return 3.25*bp_rp + 9.625

wd_obj = []
ms_obj = []
drop = []

for i in tqdm (range(len(catalog))):
    mg1 = wd_sep(catalog['bp_rp1'][i])
    mg2 = wd_sep(catalog['bp_rp2'][i])
    
    M1 = catalog['phot_g_mean_mag1'][i] + 5 * (np.log10(catalog['parallax1'][i] / 100))
    M2 = catalog['phot_g_mean_mag2'][i] + 5 * (np.log10(catalog['parallax2'][i] / 100))
    
    if M1 > mg1 and M2 < mg2:
        wd_obj.append(1)
        ms_obj.append(2)
    elif M2 > mg2 and M1 < mg1:
        wd_obj.append(2)
        ms_obj.append(1)
    else:
        drop.append(i)
               
catalog.remove_rows(drop)
catalog['wd_obj'] = wd_obj
catalog['ms_obj'] = ms_obj

100%|██████████████████████████████████████████████████████████████████████████| 22563/22563 [00:00<00:00, 39969.61it/s]


In [10]:
def separate(catalog, column, ms_obj, wd_obj, newname = ''):      
    mstemp_arr = [ catalog[str(column) + str(ms_obj[i])][i] for i in range(len(ms_obj)) ]
    wdtemp_arr = [ catalog[str(column) + str(wd_obj[i])][i] for i in range(len(wd_obj)) ]
    
    catalog['ms_' + str(column)] = mstemp_arr
    catalog['wd_' + str(column)] = wdtemp_arr
    return catalog

### ---

convert_cols = ['source_id', 'parallax', 'parallax_over_error', 'phot_g_mean_mag', 'phot_g_mean_flux',
                'phot_g_mean_flux_error', 'phot_bp_mean_mag', 'phot_bp_mean_flux', 'phot_bp_mean_flux_error',
                'phot_rp_mean_mag', 'phot_rp_mean_flux', 'phot_rp_mean_flux_error', 'bp_rp', 'ra', 'ra_error', 'dec', 'dec_error', 'l', 'b']

for col in convert_cols:
    catalog = separate(catalog, col, ms_obj, wd_obj)
    
catalog['wd_m_g'] = catalog['wd_phot_g_mean_mag'] + 5 * np.log10(catalog['wd_parallax'] / 100)
catalog['ms_m_g'] = catalog['ms_phot_g_mean_mag'] + 5 * np.log10(catalog['ms_parallax'] / 100)

In [11]:
from astropy.table import unique, Table

wd_c1['wd_pos'] = SkyCoord(wd_c1['FIBER_RA'], wd_c1['FIBER_DEC'], unit='deg')
ms_c1['ms_pos'] = SkyCoord(ms_c1['FIBER_RA'], ms_c1['FIBER_DEC'], unit='deg')

catalog['wd_pos'] = SkyCoord(catalog['wd_ra'], catalog['wd_dec'], unit='deg')
catalog['ms_pos'] = SkyCoord(catalog['ms_ra'], catalog['ms_dec'], unit='deg')

In [12]:
wd_c1[['wd_pos']]

wd_pos
"deg,deg"
SkyCoord
"35.103846,-6.2921282"
"35.01667,-6.2686349"
"34.90911,-6.0668356"
"34.972713,-6.1898776"
"34.991105,-6.0930068"
"35.056275,-6.0171515"
"35.407754,-6.4484517"
"35.198357,-6.2015798"


In [13]:
from inspect import getmembers, isfunction
from astropy import table

print(getmembers(table, isfunction))

[('dstack', <function dstack at 0x7faa8839fa60>), ('hstack', <function hstack at 0x7faa8839fb80>), ('join', <function join at 0x7faa8839f940>), ('join_distance', <function join_distance at 0x7faa8839f8b0>), ('join_skycoord', <function join_skycoord at 0x7faa8839f820>), ('represent_mixins_as_columns', <function represent_mixins_as_columns at 0x7faa8835cd30>), ('setdiff', <function setdiff at 0x7faa8839f9d0>), ('unique', <function unique at 0x7faa8839fc10>), ('vstack', <function vstack at 0x7faa8839faf0>)]


In [14]:
import astropy

print(astropy.__version__)

5.1


In [15]:
from astropy.table import join_skycoord

# Join WDs in SDSS-5
wd_catalog = unique(join(catalog, wd_c1, join_funcs={'wd_pos': join_skycoord(1 * u.arcsecond)}), keys = 'wd_pos_id')
for key in wd_c1.keys():
    if key != 'wd_pos':
        wd_catalog.rename_column(key, 'wd_' + key)

# Join MS in SDSS-5
ms_catalog = unique(join(catalog, ms_c1, join_funcs={'ms_pos': join_skycoord(1 * u.arcsecond)}), keys = 'ms_pos_id')
for key in ms_c1.keys():
    if key != 'ms_pos':
        ms_catalog.rename_column(key, 'ms_' + key)

for key in catalog.keys():
    if key != 'ms_source_id':
        try:
            ms_catalog.remove_column(key)
        except:
            ms_catalog.remove_column(key + '_1')
            ms_catalog.rename_column(key + '_2', key)
ms_sdss5_wd_sdss5 = join(wd_catalog, ms_catalog, keys = 'ms_source_id')

In [16]:
ms_sdss5_wd_sdss5

wd_pos_id,solution_id1,solution_id2,source_id1,source_id2,random_index1,random_index2,ref_epoch1,ref_epoch2,ra1,ra2,ra_error1,ra_error2,dec1,dec2,dec_error1,dec_error2,parallax1,parallax2,parallax_error1,parallax_error2,parallax_over_error1,parallax_over_error2,pm1,pm2,pmra1,pmra2,pmra_error1,pmra_error2,pmdec1,pmdec2,pmdec_error1,pmdec_error2,ra_dec_corr1,ra_dec_corr2,ra_parallax_corr1,ra_parallax_corr2,ra_pmra_corr1,ra_pmra_corr2,ra_pmdec_corr1,ra_pmdec_corr2,dec_parallax_corr1,dec_parallax_corr2,dec_pmra_corr1,dec_pmra_corr2,dec_pmdec_corr1,dec_pmdec_corr2,parallax_pmra_corr1,parallax_pmra_corr2,parallax_pmdec_corr1,parallax_pmdec_corr2,pmra_pmdec_corr1,pmra_pmdec_corr2,astrometric_n_obs_al1,astrometric_n_obs_al2,astrometric_n_obs_ac1,astrometric_n_obs_ac2,astrometric_n_good_obs_al1,astrometric_n_good_obs_al2,astrometric_n_bad_obs_al1,astrometric_n_bad_obs_al2,astrometric_gof_al1,astrometric_gof_al2,astrometric_chi2_al1,astrometric_chi2_al2,astrometric_excess_noise1,astrometric_excess_noise2,astrometric_excess_noise_sig1,astrometric_excess_noise_sig2,astrometric_params_solved1,astrometric_params_solved2,astrometric_primary_flag1,astrometric_primary_flag2,nu_eff_used_in_astrometry1,nu_eff_used_in_astrometry2,pseudocolour1,pseudocolour2,pseudocolour_error1,pseudocolour_error2,ra_pseudocolour_corr1,ra_pseudocolour_corr2,dec_pseudocolour_corr1,dec_pseudocolour_corr2,parallax_pseudocolour_corr1,parallax_pseudocolour_corr2,pmra_pseudocolour_corr1,pmra_pseudocolour_corr2,pmdec_pseudocolour_corr1,pmdec_pseudocolour_corr2,astrometric_matched_transits1,astrometric_matched_transits2,visibility_periods_used1,visibility_periods_used2,astrometric_sigma5d_max1,astrometric_sigma5d_max2,matched_transits1,matched_transits2,new_matched_transits1,new_matched_transits2,matched_transits_removed1,matched_transits_removed2,ipd_gof_harmonic_amplitude1,ipd_gof_harmonic_amplitude2,ipd_gof_harmonic_phase1,ipd_gof_harmonic_phase2,ipd_frac_multi_peak1,ipd_frac_multi_peak2,ipd_frac_odd_win1,ipd_frac_odd_win2,ruwe1,ruwe2,scan_direction_strength_k11,scan_direction_strength_k12,scan_direction_strength_k21,scan_direction_strength_k22,scan_direction_strength_k31,scan_direction_strength_k32,scan_direction_strength_k41,scan_direction_strength_k42,scan_direction_mean_k11,scan_direction_mean_k12,scan_direction_mean_k21,scan_direction_mean_k22,scan_direction_mean_k31,scan_direction_mean_k32,scan_direction_mean_k41,scan_direction_mean_k42,duplicated_source1,duplicated_source2,phot_g_n_obs1,phot_g_n_obs2,phot_g_mean_flux1,phot_g_mean_flux2,phot_g_mean_flux_error1,phot_g_mean_flux_error2,phot_g_mean_flux_over_error1,phot_g_mean_flux_over_error2,phot_g_mean_mag1,phot_g_mean_mag2,phot_bp_n_obs1,phot_bp_n_obs2,phot_bp_mean_flux1,phot_bp_mean_flux2,phot_bp_mean_flux_error1,phot_bp_mean_flux_error2,phot_bp_mean_flux_over_error1,phot_bp_mean_flux_over_error2,phot_bp_mean_mag1,phot_bp_mean_mag2,phot_rp_n_obs1,phot_rp_n_obs2,phot_rp_mean_flux1,phot_rp_mean_flux2,phot_rp_mean_flux_error1,phot_rp_mean_flux_error2,phot_rp_mean_flux_over_error1,phot_rp_mean_flux_over_error2,phot_rp_mean_mag1,phot_rp_mean_mag2,phot_bp_n_contaminated_transits1,phot_bp_n_contaminated_transits2,phot_bp_n_blended_transits1,phot_bp_n_blended_transits2,phot_rp_n_contaminated_transits1,phot_rp_n_contaminated_transits2,phot_rp_n_blended_transits1,phot_rp_n_blended_transits2,phot_proc_mode1,phot_proc_mode2,phot_bp_rp_excess_factor1,phot_bp_rp_excess_factor2,bp_rp1,bp_rp2,bp_g1,bp_g2,g_rp1,g_rp2,dr2_radial_velocity1,dr2_radial_velocity2,dr2_radial_velocity_error1,dr2_radial_velocity_error2,dr2_rv_nb_transits1,dr2_rv_nb_transits2,dr2_rv_template_teff1,dr2_rv_template_teff2,dr2_rv_template_logg1,dr2_rv_template_logg2,dr2_rv_template_fe_h1,dr2_rv_template_fe_h2,l1,l2,b1,b2,ecl_lon1,ecl_lon2,ecl_lat1,ecl_lat2,pairdistance,sep_AU,binary_type,Sigma18,R_chance_align,dr2_source_id1,dr2_source_id2,dr2_parallax1,dr2_parallax2,dr2_parallax_error1,dr2_parallax_error2,dr2_pmra1,dr2_pmra2,dr2_pmdec1,dr2_pmdec2,dr2_pmra_

In [17]:
from astropy.table import Table, vstack, hstack

from astroquery.sdss import SDSS
from astroquery.gaia import Gaia
from astropy import constants as c

# Fetch targets with MS RVs in Gaia

ADQL_CODE1 = """SELECT sdss.original_ext_source_id as ms_bestobjid, gaia.source_id as ms_source_id, gaia.radial_velocity, gaia.radial_velocity_error
    FROM gaiadr3.gaia_source as gaia
    JOIN gaiaedr3.sdssdr13_best_neighbour as sdss
    ON gaia.source_id = sdss.source_id      
    WHERE gaia.source_id in {} AND
    radial_velocity IS NOT null AND
    radial_velocity_error < 7""".format(tuple(wd_catalog['ms_source_id']))
d1 = (Gaia.launch_job(ADQL_CODE1,dump_to_file=False).get_results())
ms_gaia_wd_sdss5 = join(wd_catalog, d1, keys = 'ms_source_id')
    
# Fetch targets with MS spectra in SDSS-4

ADQL_CODE1 = """SELECT sdss.original_ext_source_id as ms_bestobjid, gaia.source_id as ms_source_id, gaia.radial_velocity, gaia.radial_velocity_error
    FROM gaiadr3.gaia_source as gaia
    JOIN gaiaedr3.sdssdr13_best_neighbour as sdss
    ON gaia.source_id = sdss.source_id      
    WHERE gaia.source_id in {}""".format(tuple(wd_catalog['ms_source_id']))
gd2 = Gaia.launch_job(ADQL_CODE1,dump_to_file=False).get_results() 

stardats = []
iters = len(gd2) // 100

for i in tqdm(range(iters)):
    SDSS_QUERY = """select sp.bestObjID, sp.plate, sp.mjd, sp.fiberID, sp.instrument 
        from dbo.SpecObjAll as sp
        where sp.bestObjID in {}""".format(tuple(gd2['ms_bestobjid'][100*i:100*i+100]))
    
    f = SDSS.query_sql(SDSS_QUERY)
    if f is not None:
        stardats.append(SDSS.query_sql(SDSS_QUERY))

d2 = vstack(stardats)
d2.rename_column('bestObjID', 'bestobjid')
d2.rename_column('fiberID', 'fiberid')

print(d2.keys())

for key in d2.keys():
    d2.rename_column(key, 'ms_' + key)

d2 = join(d2, gd2, keys = 'ms_bestobjid')
ms_sdss4_wd_sdss5 = join(wd_catalog, d2, keys = 'ms_source_id')
#except:
#    print('No objects with MS spectra in SDSS-4 and WD spectra in SDSS-5')
#    ms_sdss4_wd_sdss5 = [0]
#ms_in_sdss4 = unique(d2, keys = 'wd_bestobjid')

  0%|                                                                                             | 0/6 [00:00<?, ?it/s]/home/arseneau/anaconda3/lib/python3.9/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|█████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.49it/s]

['bestobjid', 'plate', 'mjd', 'fiberid', 'instrument']


In [18]:
ms_sdss5_wd_sdss4 = Table.read('sdss5/temp_ms_sdss5_wd_sdss4.fits')
ms_sdss5_wd_sdss4['ms_pos'] = SkyCoord(ms_sdss5_wd_sdss4['ms_ra'], ms_sdss5_wd_sdss4['ms_dec'], unit='deg')

ms_sdss5_wd_sdss4 = unique(join(ms_sdss5_wd_sdss4, ms_c1, join_funcs={'ms_pos': join_skycoord(1 * u.arcsecond)}), keys = 'ms_pos_id')
for key in ms_c1.keys():
    if key != 'ms_pos':
        ms_sdss5_wd_sdss4.rename_column(key, 'ms_' + key)

In [19]:
ms_sdss5_wd_sdss4

ms_pos_id,solution_id1,solution_id2,source_id1,source_id2,random_index1,random_index2,ref_epoch1,ref_epoch2,ra1,ra2,ra_error1,ra_error2,dec1,dec2,dec_error1,dec_error2,parallax1,parallax2,parallax_error1,parallax_error2,parallax_over_error1,parallax_over_error2,pm1,pm2,pmra1,pmra2,pmra_error1,pmra_error2,pmdec1,pmdec2,pmdec_error1,pmdec_error2,ra_dec_corr1,ra_dec_corr2,ra_parallax_corr1,ra_parallax_corr2,ra_pmra_corr1,ra_pmra_corr2,ra_pmdec_corr1,ra_pmdec_corr2,dec_parallax_corr1,dec_parallax_corr2,dec_pmra_corr1,dec_pmra_corr2,dec_pmdec_corr1,dec_pmdec_corr2,parallax_pmra_corr1,parallax_pmra_corr2,parallax_pmdec_corr1,parallax_pmdec_corr2,pmra_pmdec_corr1,pmra_pmdec_corr2,astrometric_n_obs_al1,astrometric_n_obs_al2,astrometric_n_obs_ac1,astrometric_n_obs_ac2,astrometric_n_good_obs_al1,astrometric_n_good_obs_al2,astrometric_n_bad_obs_al1,astrometric_n_bad_obs_al2,astrometric_gof_al1,astrometric_gof_al2,astrometric_chi2_al1,astrometric_chi2_al2,astrometric_excess_noise1,astrometric_excess_noise2,astrometric_excess_noise_sig1,astrometric_excess_noise_sig2,astrometric_params_solved1,astrometric_params_solved2,astrometric_primary_flag1,astrometric_primary_flag2,nu_eff_used_in_astrometry1,nu_eff_used_in_astrometry2,pseudocolour1,pseudocolour2,pseudocolour_error1,pseudocolour_error2,ra_pseudocolour_corr1,ra_pseudocolour_corr2,dec_pseudocolour_corr1,dec_pseudocolour_corr2,parallax_pseudocolour_corr1,parallax_pseudocolour_corr2,pmra_pseudocolour_corr1,pmra_pseudocolour_corr2,pmdec_pseudocolour_corr1,pmdec_pseudocolour_corr2,astrometric_matched_transits1,astrometric_matched_transits2,visibility_periods_used1,visibility_periods_used2,astrometric_sigma5d_max1,astrometric_sigma5d_max2,matched_transits1,matched_transits2,new_matched_transits1,new_matched_transits2,matched_transits_removed1,matched_transits_removed2,ipd_gof_harmonic_amplitude1,ipd_gof_harmonic_amplitude2,ipd_gof_harmonic_phase1,ipd_gof_harmonic_phase2,ipd_frac_multi_peak1,ipd_frac_multi_peak2,ipd_frac_odd_win1,ipd_frac_odd_win2,ruwe1,ruwe2,scan_direction_strength_k11,scan_direction_strength_k12,scan_direction_strength_k21,scan_direction_strength_k22,scan_direction_strength_k31,scan_direction_strength_k32,scan_direction_strength_k41,scan_direction_strength_k42,scan_direction_mean_k11,scan_direction_mean_k12,scan_direction_mean_k21,scan_direction_mean_k22,scan_direction_mean_k31,scan_direction_mean_k32,scan_direction_mean_k41,scan_direction_mean_k42,duplicated_source1,duplicated_source2,phot_g_n_obs1,phot_g_n_obs2,phot_g_mean_flux1,phot_g_mean_flux2,phot_g_mean_flux_error1,phot_g_mean_flux_error2,phot_g_mean_flux_over_error1,phot_g_mean_flux_over_error2,phot_g_mean_mag1,phot_g_mean_mag2,phot_bp_n_obs1,phot_bp_n_obs2,phot_bp_mean_flux1,phot_bp_mean_flux2,phot_bp_mean_flux_error1,phot_bp_mean_flux_error2,phot_bp_mean_flux_over_error1,phot_bp_mean_flux_over_error2,phot_bp_mean_mag1,phot_bp_mean_mag2,phot_rp_n_obs1,phot_rp_n_obs2,phot_rp_mean_flux1,phot_rp_mean_flux2,phot_rp_mean_flux_error1,phot_rp_mean_flux_error2,phot_rp_mean_flux_over_error1,phot_rp_mean_flux_over_error2,phot_rp_mean_mag1,phot_rp_mean_mag2,phot_bp_n_contaminated_transits1,phot_bp_n_contaminated_transits2,phot_bp_n_blended_transits1,phot_bp_n_blended_transits2,phot_rp_n_contaminated_transits1,phot_rp_n_contaminated_transits2,phot_rp_n_blended_transits1,phot_rp_n_blended_transits2,phot_proc_mode1,phot_proc_mode2,phot_bp_rp_excess_factor1,phot_bp_rp_excess_factor2,bp_rp1,bp_rp2,bp_g1,bp_g2,g_rp1,g_rp2,dr2_radial_velocity1,dr2_radial_velocity2,dr2_radial_velocity_error1,dr2_radial_velocity_error2,dr2_rv_nb_transits1,dr2_rv_nb_transits2,dr2_rv_template_teff1,dr2_rv_template_teff2,dr2_rv_template_logg1,dr2_rv_template_logg2,dr2_rv_template_fe_h1,dr2_rv_template_fe_h2,l1,l2,b1,b2,ecl_lon1,ecl_lon2,ecl_lat1,ecl_lat2,pairdistance,sep_AU,binary_type,Sigma18,R_chance_align,dr2_source_id1,dr2_source_id2,dr2_parallax1,dr2_parallax2,dr2_parallax_error1,dr2_parallax_error2,dr2_pmra1,dr2_pmra2,dr2_pmdec1,dr2_pmdec2,dr2_pmra_

In [20]:
# MS Gaia, WD SDSS-V

ms_gaia_wd_sdss5['wd_filepath'] =  ms_gaia_wd_sdss5['wd_SPEC_FILE']
ms_gaia_wd_sdss5['wd_plate'] = ms_gaia_wd_sdss5['wd_FIELD']
ms_gaia_wd_sdss5['wd_mjd'] = ms_gaia_wd_sdss5['wd_MJD']
ms_gaia_wd_sdss5['wd_fiberid'] = ms_gaia_wd_sdss5['wd_CATALOGID']
ms_gaia_wd_sdss5['wd_instrument'] = ms_gaia_wd_sdss5['wd_OBS']

ms_gaia_wd_sdss5['uses_sdss5'] = np.ones(len(ms_gaia_wd_sdss5))
ms_gaia_wd_sdss5['wd_rv_from'] = ['sdss5' for i in range(len(ms_gaia_wd_sdss5))]
ms_gaia_wd_sdss5['ms_rv_from'] = ['gaia' for i in range(len(ms_gaia_wd_sdss5))]
ms_gaia_wd_sdss5['ms_instrument'] = ['gaia' for i in range(len(ms_gaia_wd_sdss5))]


ms_gaia_wd_sdss5['wd_rv'] = 999*np.ones(len(ms_gaia_wd_sdss5))
ms_gaia_wd_sdss5['wd_e_rv'] = 999*np.ones(len(ms_gaia_wd_sdss5))

ms_gaia_wd_sdss5 = unique(ms_gaia_wd_sdss5, keys = 'wd_source_id')

# MS SDSS-IV, WD SDSS-V

ms_sdss4_wd_sdss5['wd_filepath'] =  ms_sdss4_wd_sdss5['wd_SPEC_FILE']
ms_sdss4_wd_sdss5['wd_plate'] = ms_sdss4_wd_sdss5['wd_FIELD']
ms_sdss4_wd_sdss5['wd_mjd'] = ms_sdss4_wd_sdss5['wd_MJD']
ms_sdss4_wd_sdss5['wd_fiberid'] = ms_sdss4_wd_sdss5['wd_CATALOGID']
ms_sdss4_wd_sdss5['wd_instrument'] = ms_sdss4_wd_sdss5['wd_OBS']

ms_sdss4_wd_sdss5['uses_sdss5'] = np.ones(len(ms_sdss4_wd_sdss5))
ms_sdss4_wd_sdss5['wd_rv_from'] = ['sdss5' for i in range(len(ms_sdss4_wd_sdss5))]
ms_sdss4_wd_sdss5['ms_rv_from'] = ['sdss4' for i in range(len(ms_sdss4_wd_sdss5))]


ms_sdss4_wd_sdss5['wd_rv'] = 999*np.ones(len(ms_sdss4_wd_sdss5))
ms_sdss4_wd_sdss5['wd_e_rv'] = 999*np.ones(len(ms_sdss4_wd_sdss5))

ms_sdss4_wd_sdss5 = unique(ms_sdss4_wd_sdss5, keys = 'wd_source_id')

# MS SDSS-V, WD SDSS-V

ms_sdss5_wd_sdss5['ms_filepath'] =  ms_sdss5_wd_sdss5['ms_SPEC_FILE']
ms_sdss5_wd_sdss5['ms_plate'] = ms_sdss5_wd_sdss5['ms_FIELD']
ms_sdss5_wd_sdss5['ms_mjd'] = ms_sdss5_wd_sdss5['ms_MJD']
ms_sdss5_wd_sdss5['ms_fiberid'] = ms_sdss5_wd_sdss5['wd_CATALOGID']
ms_sdss5_wd_sdss5['ms_instrument'] = ms_sdss5_wd_sdss5['ms_OBS']

ms_sdss5_wd_sdss5['wd_filepath'] =  ms_sdss5_wd_sdss5['wd_SPEC_FILE']
ms_sdss5_wd_sdss5['wd_plate'] = ms_sdss5_wd_sdss5['wd_FIELD']
ms_sdss5_wd_sdss5['wd_mjd'] = ms_sdss5_wd_sdss5['wd_MJD']
ms_sdss5_wd_sdss5['wd_fiberid'] = ms_sdss5_wd_sdss5['wd_CATALOGID']
ms_sdss5_wd_sdss5['wd_instrument'] = ms_sdss5_wd_sdss5['wd_OBS']

ms_sdss5_wd_sdss5['wd_rv'] = 999*np.ones(len(ms_sdss5_wd_sdss5))
ms_sdss5_wd_sdss5['wd_e_rv'] = 999*np.ones(len(ms_sdss5_wd_sdss5))

ms_sdss5_wd_sdss5['uses_sdss5'] = np.ones(len(ms_sdss5_wd_sdss5))
ms_sdss5_wd_sdss5['wd_rv_from'] = ['sdss5' for i in range(len(ms_sdss5_wd_sdss5))]
ms_sdss5_wd_sdss5['ms_rv_from'] = ['sdss5' for i in range(len(ms_sdss5_wd_sdss5))]

ms_sdss5_wd_sdss5 = unique(ms_sdss5_wd_sdss5, keys = 'wd_source_id')

# MS SDSS-V, WD SDSS-IV

ms_sdss5_wd_sdss4['ms_filepath'] =  ms_sdss5_wd_sdss4['ms_SPEC_FILE']
ms_sdss5_wd_sdss4['ms_plate'] = ms_sdss5_wd_sdss4['ms_FIELD']
ms_sdss5_wd_sdss4['ms_mjd'] = ms_sdss5_wd_sdss4['ms_MJD']
ms_sdss5_wd_sdss4['ms_fiberid'] = ms_sdss5_wd_sdss4['ms_CATALOGID']
ms_sdss5_wd_sdss4['ms_instrument'] = ms_sdss5_wd_sdss4['ms_OBS']

ms_sdss5_wd_sdss4['wd_rv'] = 999*np.ones(len(ms_sdss5_wd_sdss4))
ms_sdss5_wd_sdss4['wd_e_rv'] = 999*np.ones(len(ms_sdss5_wd_sdss4))

ms_sdss5_wd_sdss4['uses_sdss5'] = np.ones(len(ms_sdss5_wd_sdss4))
ms_sdss5_wd_sdss4['wd_rv_from'] = ['sdss4' for i in range(len(ms_sdss5_wd_sdss4))]
ms_sdss5_wd_sdss4['ms_rv_from'] = ['sdss5' for i in range(len(ms_sdss5_wd_sdss4))]

ms_sdss5_wd_sdss4 = unique(ms_sdss5_wd_sdss4, keys = 'wd_source_id')

# Write Catalogs

ms_gaia_wd_sdss5.write('sdss5/ms_gaia_wd_sdss5.fits', overwrite = True)
ms_sdss4_wd_sdss5.write('sdss5/ms_sdss4_wd_sdss5.fits', overwrite = True)
ms_sdss5_wd_sdss5.write('sdss5/ms_sdss5_wd_sdss5.fits', overwrite = True)
ms_sdss5_wd_sdss4.write('sdss5/ms_sdss5_wd_sdss4.fits', overwrite = True)

In [21]:
print(len(ms_gaia_wd_sdss5))
print(len(unique(ms_gaia_wd_sdss5, keys = 'wd_source_id')))

print(len(ms_sdss4_wd_sdss5))
print(len(unique(ms_sdss4_wd_sdss5, keys = 'wd_source_id')))

print(len(ms_sdss5_wd_sdss5))
print(len(unique(ms_sdss5_wd_sdss5, keys = 'wd_source_id')))

print(len(ms_sdss5_wd_sdss4))
print(len(unique(ms_sdss5_wd_sdss4, keys = 'wd_source_id')))

209
209
5
5
88
88
71
71


In [22]:
ms_sdss4_wd_sdss5[ms_sdss4_wd_sdss5['ms_source_id'] == 875615158730751488]

wd_pos_id,solution_id1,solution_id2,source_id1,source_id2,random_index1,random_index2,ref_epoch1,ref_epoch2,ra1,ra2,ra_error1,ra_error2,dec1,dec2,dec_error1,dec_error2,parallax1,parallax2,parallax_error1,parallax_error2,parallax_over_error1,parallax_over_error2,pm1,pm2,pmra1,pmra2,pmra_error1,pmra_error2,pmdec1,pmdec2,pmdec_error1,pmdec_error2,ra_dec_corr1,ra_dec_corr2,ra_parallax_corr1,ra_parallax_corr2,ra_pmra_corr1,ra_pmra_corr2,ra_pmdec_corr1,ra_pmdec_corr2,dec_parallax_corr1,dec_parallax_corr2,dec_pmra_corr1,dec_pmra_corr2,dec_pmdec_corr1,dec_pmdec_corr2,parallax_pmra_corr1,parallax_pmra_corr2,parallax_pmdec_corr1,parallax_pmdec_corr2,pmra_pmdec_corr1,pmra_pmdec_corr2,astrometric_n_obs_al1,astrometric_n_obs_al2,astrometric_n_obs_ac1,astrometric_n_obs_ac2,astrometric_n_good_obs_al1,astrometric_n_good_obs_al2,astrometric_n_bad_obs_al1,astrometric_n_bad_obs_al2,astrometric_gof_al1,astrometric_gof_al2,astrometric_chi2_al1,astrometric_chi2_al2,astrometric_excess_noise1,astrometric_excess_noise2,astrometric_excess_noise_sig1,astrometric_excess_noise_sig2,astrometric_params_solved1,astrometric_params_solved2,astrometric_primary_flag1,astrometric_primary_flag2,nu_eff_used_in_astrometry1,nu_eff_used_in_astrometry2,pseudocolour1,pseudocolour2,pseudocolour_error1,pseudocolour_error2,ra_pseudocolour_corr1,ra_pseudocolour_corr2,dec_pseudocolour_corr1,dec_pseudocolour_corr2,parallax_pseudocolour_corr1,parallax_pseudocolour_corr2,pmra_pseudocolour_corr1,pmra_pseudocolour_corr2,pmdec_pseudocolour_corr1,pmdec_pseudocolour_corr2,astrometric_matched_transits1,astrometric_matched_transits2,visibility_periods_used1,visibility_periods_used2,astrometric_sigma5d_max1,astrometric_sigma5d_max2,matched_transits1,matched_transits2,new_matched_transits1,new_matched_transits2,matched_transits_removed1,matched_transits_removed2,ipd_gof_harmonic_amplitude1,ipd_gof_harmonic_amplitude2,ipd_gof_harmonic_phase1,ipd_gof_harmonic_phase2,ipd_frac_multi_peak1,ipd_frac_multi_peak2,ipd_frac_odd_win1,ipd_frac_odd_win2,ruwe1,ruwe2,scan_direction_strength_k11,scan_direction_strength_k12,scan_direction_strength_k21,scan_direction_strength_k22,scan_direction_strength_k31,scan_direction_strength_k32,scan_direction_strength_k41,scan_direction_strength_k42,scan_direction_mean_k11,scan_direction_mean_k12,scan_direction_mean_k21,scan_direction_mean_k22,scan_direction_mean_k31,scan_direction_mean_k32,scan_direction_mean_k41,scan_direction_mean_k42,duplicated_source1,duplicated_source2,phot_g_n_obs1,phot_g_n_obs2,phot_g_mean_flux1,phot_g_mean_flux2,phot_g_mean_flux_error1,phot_g_mean_flux_error2,phot_g_mean_flux_over_error1,phot_g_mean_flux_over_error2,phot_g_mean_mag1,phot_g_mean_mag2,phot_bp_n_obs1,phot_bp_n_obs2,phot_bp_mean_flux1,phot_bp_mean_flux2,phot_bp_mean_flux_error1,phot_bp_mean_flux_error2,phot_bp_mean_flux_over_error1,phot_bp_mean_flux_over_error2,phot_bp_mean_mag1,phot_bp_mean_mag2,phot_rp_n_obs1,phot_rp_n_obs2,phot_rp_mean_flux1,phot_rp_mean_flux2,phot_rp_mean_flux_error1,phot_rp_mean_flux_error2,phot_rp_mean_flux_over_error1,phot_rp_mean_flux_over_error2,phot_rp_mean_mag1,phot_rp_mean_mag2,phot_bp_n_contaminated_transits1,phot_bp_n_contaminated_transits2,phot_bp_n_blended_transits1,phot_bp_n_blended_transits2,phot_rp_n_contaminated_transits1,phot_rp_n_contaminated_transits2,phot_rp_n_blended_transits1,phot_rp_n_blended_transits2,phot_proc_mode1,phot_proc_mode2,phot_bp_rp_excess_factor1,phot_bp_rp_excess_factor2,bp_rp1,bp_rp2,bp_g1,bp_g2,g_rp1,g_rp2,dr2_radial_velocity1,dr2_radial_velocity2,dr2_radial_velocity_error1,dr2_radial_velocity_error2,dr2_rv_nb_transits1,dr2_rv_nb_transits2,dr2_rv_template_teff1,dr2_rv_template_teff2,dr2_rv_template_logg1,dr2_rv_template_logg2,dr2_rv_template_fe_h1,dr2_rv_template_fe_h2,l1,l2,b1,b2,ecl_lon1,ecl_lon2,ecl_lat1,ecl_lat2,pairdistance,sep_AU,binary_type,Sigma18,R_chance_align,dr2_source_id1,dr2_source_id2,dr2_parallax1,dr2_parallax2,dr2_parallax_error1,dr2_parallax_error2,dr2_pmra1,dr2_pmra2,dr2_pmdec1,dr2_pmdec2,dr2_pmra_

In [23]:
print(ms_sdss5_wd_sdss4[ms_sdss5_wd_sdss4['ms_source_id'] == 875615158730751488])

ms_pos_id solution_id1 solution_id2 source_id1 source_id2 ... wd_rv wd_e_rv uses_sdss5 wd_rv_from ms_rv_from
                                                          ...                                               
--------- ------------ ------------ ---------- ---------- ... ----- ------- ---------- ---------- ----------
